# Outline

* [Introduction](#intro)
* [What is this competition about?](#what)
* [Date Preparation](#data)
    - [Fastai's High Level Segmentation API](#data-one)
    - [Fastai's Mid Level Segmentation API (Data Block)](#data-two)
    - [Fastai low level API](#data-three)
    - [2.5D Data with Fastai low level API](#data-four)
    - [2.5D Data pre-padded with fastai augmentations and random resized crops](#data-five)
    - [2.5D Data with albumentations](#data-six)
* [Training](#train)
    - [Metrics and Losses](#train-one)
    - [Using SMP models with Fastai](#train-two)
    - [Using Fastai's DynamicUnet with timm encoder?](#train-three)
* [Inference](#infer)

<a id="intro"></a>

# Introduction

In this notebook, I'll go through the creating a pipeline for this competition step by step using fastai, and I'll incorporate the various methods that participants have shared during the past also using fastai.

If you are beginner don't worry, I'll try to walk you through easy steps to understand what is going on. And if you are here just to see how specific ideas are implemented using fastai's api, like multi-channels 2.5D data or custom albumentations augmentation, then I hope you find what you are looking for in the right depth that you need.

<a id="what"></a>
# What is this competition about?

For this part, I won't reinvent the wheel trying to do an in-depth EDA when other kagglers have made tremendous work. So please check the following notebooks to get an understanding of the problem we are adressing and the data of the competition.

[UWM - GI Tract Image Segmentation - EDA by Darien Schettler](https://www.kaggle.com/code/dschettler8845/uwm-gi-tract-image-segmentation-eda)

[⚕️ AW-Madison: EDA & In Depth Mask Exploration by Andrada Olteanu](https://www.kaggle.com/code/andradaolteanu/aw-madison-eda-in-depth-mask-exploration)

<a id="data"></a>
# Data Preparation

#### What are we supposed to do in this step exactly?
We know that the competition provides a dataframe so let's take a look at that.

In [ ]:
from fastai.vision.all import *

train = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv', low_memory=False)
train.head()

In order to get our pipeline working, we need each slice to have it's three classes segmentations (if they exist) in one row. We can do this using pandas.

In [ ]:
train_df = train.pivot(index='id', columns='class', values='segmentation').reset_index()
train_df.head()

#### What is next?

We need to get our data to work with one of fastai's api levels. 

## Fastai's High Level Segmentation API

To get this one working we need 3 things:
1. `path`: A Path object where the images reside
2. `fnames`: The filenames of the images
3. `label_func`: A function through which we can load the masks

In [ ]:
# The Path objects are really useful
# You can read about them in here https://docs.python.org/3/library/pathlib.html
path = Path('../input/uw-madison-gi-tract-image-segmentation/train')

# To get the filenames we'll use a nice helper function provided by fastai called `get_image_files` 
# that collects the filenames of all images within a Path
fnames = get_image_files(path)

# Let's inspect these so far
path, fnames

We can see that within `fnames` is a `Path` object for each image. Now let's move to create `label_func`, but before we do it let's think about what we need out of it.

1. The function should take an image `fname` as an input which is a `Path` object
2. Using this `Path`, it should construct a mask using from the rle format provided within `train_df`

That's a high level of what needs to be done, but it needs to be dissected a bit to understand it, especially the second point, let's reiterate more extensively over that.

1. The function should find a way to map the `Path` object to the it's corresponding row in `train_df`
2. Then it should convert the rle formatted masks into masks that can be overalyed with an image

#### Let's address the first part. 

#### How can we map a `Path` object be used to a row in the dataframe?

A great feature of `Path` than we can use is accessing it's parts using an attribute `parts`.

In [ ]:
fnames[0].parts

Now all we need to do is to construct the id in the dataframe of that slice from it's parts.

In [ ]:
# This is how an id looks in the dataframe
train_df.id[0]

In [ ]:
fnames[0].parts[5] + '_' + fnames[0].parts[7][:10]

#### And I think we got that one right. Now let's wrap into a function to use it later.

In [ ]:
def get_slice_id(fname):
    return fname.parts[5] + '_' + fname.parts[7][:10]

Now since we have a way to map the `Path` to the dataframe, we need a function to convert the rle formatted masks into a mask array, and to do that we'll use a helper function.

In [ ]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
# modified from: https://www.kaggle.com/inversion/run-length-decoding-quick-start
def rle_decode(mask_rle, shape, color=1):
    """ TBD
    
    Args:
        mask_rle (str): run-length as string formated (start length)
        shape (tuple of ints): (height,width) of array to return 
    
    Returns: 
        Mask (np.array)
            - 1 indicating mask
            - 0 indicating background

    """
    # Split the string by space, then convert it into a integer array
    s = np.array(mask_rle.split(), dtype=int)

    # Every even value is the start, every odd value is the "run" length
    starts = s[0::2] - 1
    lengths = s[1::2]
    ends = starts + lengths

    # The image image is actually flattened since RLE is a 1D "run"
    if len(shape)==3:
        h, w, d = shape
        img = np.zeros((h * w, d), dtype=np.float32)
    else:
        h, w = shape
        img = np.zeros((h * w,), dtype=np.float32)

    # The color here is actually just any integer you want!
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
        
    # Don't forget to change the image back to the original shape
    return img.reshape(shape)

Now we have a way to map the slice fname to it's row, and we have a way to decode rle masks into arrays, all we need to do is to make a function that wraps all of this to output a mask for a slice fname. 

For the sake of simplicity let's test it with one one mask class in order to see how we are doing, then we can move on to use all classes.

In [ ]:
# Let's start with on slice fname
fname = fnames[0]

# First we need to get the slice row
slice_id = get_slice_id(fname)
slice_row = train_df.query('id == @slice_id')

# Then we need to extract the slice height and width which are provided in the fname last part
h, w = map(lambda x: int(x), fname.parts[-1].split('_')[2:4])
h, w

In [ ]:
# Now we create an empty numpy array with the dimensions of the slice and decode the rle mask into it
mask = np.zeros((h, w, 1))

# If the segmentation mask is not null then we shall populate the mask
if not np.isnan(slice_row['large_bowel'].item()):
    mask[:, :, 1] = rle_decode(slice_row['large_bowel'].item(), shape=(h, w), color=255)

#### I guess this now works, let's package it into a function and test making a dataloader.

In [ ]:
def label_func(fname):
    # First we need to get the slice row
    slice_id = get_slice_id(fname)
    slice_row = train_df.query('id == @slice_id')
    
    # Then we need to extract the slice width and height which are provided in the fname last part
    # Typically the height is the first part of a dimension, but for some reason the slices have
    # widths provided first
    w, h = map(lambda x: int(x), fname.parts[-1].split('_')[2:4])
    
    # Create mask array (It needs to have 3 channels but fastai will only keep the first one anyways)
    mask = np.zeros((h, w, 3), dtype=np.uint8)
    
    # If the segmentation mask is str
    if isinstance(slice_row['large_bowel'].item(), str):
        # Right now the pixel having the mask will have a code of one
        mask[:, :, 0] = rle_decode(slice_row['large_bowel'].item(), shape=(h, w), color=1)
            
    return mask

In [ ]:
# I have added Resize transformation in order to be able to collate batch of slices with different dimensions together
# It isn't the most optimal way to do it like this and we'll see why as we go
dls = SegmentationDataLoaders.from_label_func(path, fnames, label_func, item_tfms=Resize(224))

#### And now let's visualize a batch to see what's going on...

In [ ]:
dls.show_batch()

And voila, we've got ourselves a dataloader that can load single slices with their masks and can overlay them into visulaizations with little effort.

#### However, now you might wonder about the blueness of the plots, and what if we want to add the other masks? And that's exactly correct to think about!

While this dataloader served as an intro to see how we can hack a way to make a segmentation dataloader, it still isn't the best way. You can probably still train a model using this dataloader and even make a submission than can get you a decent score, given that you added the rest of the masks in the same channel and treated the problem as multi-class.

In order to do this, we only need to customize our `label_func` to add the rest of the masks in the same way we added the `large_bowel` mask, but we'll face the issue of overlapping masks, and then we can also just keep one the masks at random just to make this pipeline work.

In [ ]:
def label_func(fname):
    # First we need to get the slice row
    slice_id = get_slice_id(fname)
    slice_row = train_df.query('id == @slice_id')
    
    # Then we need to extract the slice width and height which are provided in the fname last part
    # Typically the height is the first part of a dimension, but for some reason the slices have
    # widths provided first
    w, h = map(lambda x: int(x), fname.parts[-1].split('_')[2:4])
    
    # Create mask array (It needs to have 3 channels but fastai will only keep the first one anyways)
    mask = np.zeros((h, w, 3), dtype=np.uint8)
    
    # If the segmentation mask is str
    # Each mask should have it's own code (color) where fastai will use them for identification
    if isinstance(slice_row['large_bowel'].item(), str):
        mask[:, :, 0] = rle_decode(slice_row['large_bowel'].item(), shape=(h, w), color=1)
        
    if isinstance(slice_row['small_bowel'].item(), str):
        mask[:, :, 0] = rle_decode(slice_row['small_bowel'].item(), shape=(h, w), color=2)
        
    # In case of overlap of first two masks
    mask = np.where(mask == 3, 1, mask)
        
    if isinstance(slice_row['stomach'].item(), str):
        mask[:, :, 0] = rle_decode(slice_row['stomach'].item(), shape=(h, w), color=3)
        
    # In case of overlap with one of the two masks
    mask = np.where(mask == 4, 1, mask)
    mask = np.where(mask == 5, 2, mask)
        
    return mask

Now let's make a dataloader.

In [ ]:
codes = {1: 'large_bowel', 2: 'small_bowel', 3: 'stomach'}
dls = SegmentationDataLoaders.from_label_func(path, fnames, label_func, codes=codes, item_tfms=Resize(224))

In [ ]:
dls.show_batch()

Since there is no legend in these plots, we can't see the different masks, so let's take a look at one batch.

In [ ]:
x, y = dls.one_batch()

Let's take a look at each count value using `torch.bincount`

In [ ]:
torch.bincount(y.flatten())

We can see that there are four values, the first is for the background, and the rest are for the 3 masks.

Now let's get it on the next part.

<a id="data-two"></a>
## Fastai's Mid Level Segmentation API (Data Block)

Through this level, we can make some customizations that we weren't able to do in the previous level.
1. We can load each slice in one channel
2. We can load the masks in multiple-channels 
3. We can customize the batch visualization

### How does it work?

It works really similar to `SegmentationDataLoaders`, in that we need to provide a set of `fnames` and a `label_func`, but we need to provide the `blocks` which will process our `fnames` and `label_func` output.

We'll also provide the type of splitter for train and validation.

Let's think about that for a moment, each `fname` is basically an image file that needs to be loaded using `PIL` for example, and each label function is an array that should be loaded also using `PIL`.

Lucky for us, fastai comes with ImageBlock and MaskBlock that carries that from us.

Let's now adjust the `label_func` to take the 3 masks into separate channels.

In [ ]:
def label_func(fname):
    # First we need to get the slice row
    slice_id = get_slice_id(fname)
    slice_row = train_df.query('id == @slice_id')
    
    # Then we need to extract the slice width and height which are provided in the fname last part
    # Typically the height is the first part of a dimension, but for some reason the slices have
    # widths provided first
    w, h = map(lambda x: int(x), fname.parts[-1].split('_')[2:4])
    
    # Create mask array (It needs to have 3 channels but fastai will only keep the first one anyways)
    mask = np.zeros((h, w, 3), dtype=np.uint8)
    
    # If the segmentation mask is str
    # Each mask should have it's own code (color) where fastai will use them for identification
    if isinstance(slice_row['large_bowel'].item(), str):
        mask[:, :, 0] = rle_decode(slice_row['large_bowel'].item(), shape=(h, w), color=255)
        
    if isinstance(slice_row['small_bowel'].item(), str):
        mask[:, :, 1] = rle_decode(slice_row['small_bowel'].item(), shape=(h, w), color=255)
        
    if isinstance(slice_row['stomach'].item(), str):
        mask[:, :, 2] = rle_decode(slice_row['stomach'].item(), shape=(h, w), color=255)
        
    return mask



In [ ]:
dblock = DataBlock(blocks=(ImageBlock(), MaskBlock()),
                           splitter=RandomSplitter(0.2),
                           get_y=label_func,
                           item_tfms=Resize(224))

dls = dblock.dataloaders(fnames)

dls.show_batch()

First of all, the blue tint of the plots is caused by the background of the mask, as the majority of pixels don't have any label. So this can be fixed if turned the mask into 3 channels.

Now fastai doesn't give us a Block that can make mutli channel masks out of the box, but we can one ourselves, or....

We can hack the `MaskBlock` itself. Let's take a look at what `MaskBlock` does.

In [ ]:
MaskBlock??

What we see in code is fastai `TransformBlock`, which handles out of the box transformations handling in the datablock api, but it doesn't seem to the problem.

We need to understand what the datablock api does for the mask, and here it is:
1. First the datablock takes an `fname` and loads its image and mask
2. It converts both of them mandatory into a Tensor using the `ToTensor` class that has specific methods image and masks
3. Then it goes on with other transformations including conversion of image tensor types to floats and added any augmentations or normalization

The problem must lie with `ToTensor`, and after taking a look in the fastai source code, I found out that `ToTensor` converts masks to single channel by default, and all we need to do is to make them take multi channel.

In [ ]:
# This was the code available in fastai
@ToTensor
def encodes(self, o:PILMask): return o._tensor_cls(image2tensor(o)[0])

# And this is how we customize it to suit our needs
@ToTensor
def encodes(self, o:PILMask): return o._tensor_cls(image2tensor(o))

Now let's test this.

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MaskBlock),
                           splitter=RandomSplitter(0.2),
                           get_y=label_func,
                           item_tfms=Resize(224))

dls = dblock.dataloaders(fnames)

dls.show_batch()

#### Voila, now we have 3 masks represented in the plots, and the slices themselves are better, but need to correct those with high contrast.

So what we'll do for this is add a method that loads the `fname` into an array and preprocesses it before passing it to the `ImageBlock`.

In [ ]:
def load_image(fname):
    img = np.array(Image.open(fname))
    img = np.interp(img, [np.min(img), np.max(img)], [0,255])
    return img

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MaskBlock),
                           splitter=RandomSplitter(0.2),
                           get_x=load_image,
                           get_y=label_func,
                           item_tfms=Resize(224))

dls = dblock.dataloaders(fnames)

dls.show_batch()

Now all we need to do is just customize the `cmap` of the plots, and for that we'll customize the `show_batch` method.

The `typedispatch` decorator together with the type hints will dispatch this method whenever we call `dls.show_batch` with the specified types, which are `TensorImage` and `TensorMask`.

In [ ]:
import matplotlib.patches as mpatches

@typedispatch
def show_batch(x:TensorImage, y:TensorMask, samples, ctxs=None, max_n=6, nrows=None, ncols=2, figsize=None, **kwargs):
    if figsize is None: figsize = (ncols*3, max_n//ncols * 3)
    if ctxs is None: ctxs = get_grid(max_n, nrows=nrows, ncols=ncols, figsize=figsize)
    for i,ctx in enumerate(ctxs): 
        x_i = x[i] / x[i].max()
        show_image(x_i, ctx=ctx, cmap='gray', **kwargs)
        show_image(y[i], ctx=ctx, cmap='Spectral_r', alpha=0.35, **kwargs)
        red_patch = mpatches.Patch(color='red', label='large_bowel')
        green_patch = mpatches.Patch(color='green', label='small_bowel')
        blue_patch = mpatches.Patch(color='blue', label='stomach')
        ctx.legend(handles=[red_patch, green_patch, blue_patch], fontsize=figsize[0]/2)

In [ ]:
dls.show_batch(nrows=2, ncols=4, figsize=(15, 7))

#### And now we have functional dataloader that can show batches easily, and has multi channels masks which can be used to make a decent baseline for this competition. 

<a id="data-three"></a>

## Fastai low level API

We have seen how the high level and mid level api works, and now it's time to get our hands dirty with the low level api, which will help us understand how the previous levels work, since they are both build upon it.

The low level api gives you the flexibility to pass a pipeline containing a series of transformations to a `Dataset` object, which will be used to create a dataloader.

Let's imagine how we can build the previous version of the dataloader with the low level api. 

1. First we need a function to load the image and mask into PIL format
2. Then we need to resize them to collate them together into batches
3. Then we will convert them to tensors 
4. Then we will convert these tensors from int to float

In [ ]:
# The list of transformations contain two lists
# 1. The first is for the X part of the dataset (images)
# 2. The second if for the Y part of the dataset (masks)
tfms = [[load_image, PILImageBW.create], [label_func, PILMask.create]]

# Then we just pass the fnames along the transformations
dsets = Datasets(fnames, tfms)

# With the DataBlock API, we didn't worry about passing item and batch transformations
# As some are implicitly used like ToTensor and IntToFloatTensor
# But in here we need to pass them explicitly
dls = dsets.dataloaders(after_item=[Resize(224), ToTensor], after_batch=[IntToFloatTensor])


# Now let's see if our visualization works
dls.show_batch(nrows=2, ncols=4, figsize=(15, 7))

Now if you get this part, you probably now understand how the previous API levels functioned. They just abstracted some the parts we did here. For example, the data blocks are supposed to give us the ease of not worrying about how to load images or masks, while the high level API doesn't let you worry about anything unless how to load the masks.

Personally, I love the low level API as it gives greater flexibility as we'll see now when we 
1. Load the data using 2.5D approach
2. Pad the images before loading
3. Add custom augmentations using albumentations

<a id='data-four'></a>

## 2.5D Data with Fastai low level API

First I'd like to thank [Awsaf](https://www.kaggle.com/awsaf49) for introducing the use of this [approach](https://www.kaggle.com/code/awsaf49/uwmgi-2-5d-stride-2-data) into this competition

The idea behind 2.5D data is to load the sequnetial slices as image channels, so the problem we are solving is still 2D semantic segmentation, but the data carries along in it more than 2D information. So it's not completely 3D, and it's still not 2D.

Let's start with using only 3 channels with 2 strides, just like the original notebook.

Now we'll start using the dataframe as input for our dataset instead of using the `fnames`, and that is because it's easier to specify the slices which be use as supplementary channels for the main slice. And we'll create two new columns
1. The first column will contain the slice `fname`
2. The second will contain a list of slices, with the main slice always in the middle, that we'll call `fnames`

In [ ]:
train_df['partial_fname'] = train_df.id
fname_df = pd.DataFrame({'partial_fname': [f'{fname.parts[-3]}_slice_{fname.parts[-1][6:10]}' for fname in fnames],
                         'fname': fnames})

train_df = train_df.merge(fname_df, on='partial_fname').drop('partial_fname', axis=1)

train_df['case_id'] = train_df.id.apply(lambda x: x.split('_')[0])
train_df['day_num'] = train_df.id.apply(lambda x: x.split('_')[1])

train_df['slice_w'] = train_df["fname"].apply(lambda x: int(str(x)[:-4].rsplit("_",4)[1]))
train_df['slice_h'] = train_df["fname"].apply(lambda x: int(str(x)[:-4].rsplit("_",4)[2]))


train_df.head()

In [ ]:
channels = 3
stride = 2
for j, i in enumerate(range(-1*(channels-channels//2-1), channels//2+1)):
    method = 'ffill'
    if i <= 0: method = 'bfill'
    train_df[f'fname_{j:02}'] = train_df.groupby(['case_id', 'day_num'])['fname'].shift(stride*-i).fillna(method=method)
    
train_df['fnames'] = train_df[[f'fname_{j:02d}' for j in range(channels)]].values.tolist()

train_df.head()

#### And now we just need a function to load this slices into one image, and make our transformations to create the dataloader. But we need to remember that now instead of passing fnames directly, we are passing rows. And that applies to the mask function too, so we'll need to change it.

In [ ]:
def load_image(fname):
    img = np.array(Image.open(fname))
    img = np.interp(img, [np.min(img), np.max(img)], [0,255])
    return img

def get_25D_image(row):
    imgs = np.zeros((row['slice_h'], row['slice_w'], len(row['fnames'])))
        
    for i, fname in enumerate(row['fnames']):
        img = load_image(fname)
        imgs[..., i] += img
    return imgs.astype(np.uint8)


def get_mask(row):
    mask = np.zeros((row['slice_h'], row['slice_w'], 3))
        
    if isinstance(row['large_bowel'], str):
        mask[..., 0] += rle_decode(row['large_bowel'], shape=(row['slice_h'], row['slice_w']), color=255)
    if isinstance(row['small_bowel'], str):
        mask[..., 1] += rle_decode(row['small_bowel'], shape=(row['slice_h'], row['slice_w']), color=255)
    if isinstance(row['stomach'], str):
        mask[..., 2] += rle_decode(row['stomach'], shape=(row['slice_h'], row['slice_w']), color=255)
        
    return mask.astype(np.uint8)

Now let's create out transormations another time and see how are dataloaders handles them.

In [ ]:
tfms = [[get_25D_image, PILImage.create],
        [get_mask, PILMask.create]]
    
dsets = Datasets(train_df, tfms)

dls = dsets.dataloaders(bs=16, after_item=[Resize(224), ToTensor], after_batch=[IntToFloatTensor])

# Now let's see if our visualization works
dls.show_batch(nrows=4, ncols=4, max_n=16, figsize=(15, 15))

#### Aaaand voila we have created a 2.5D dataloader using fastai's low level api. 

<a id='data-five'></a>
## Pre Padding 2.5D Data

When I first started working on this competition, I used fastai's `Resize` with `method=squish` since it enabled easy resizing of masks without worrying about the different dimensions of slices. 

The idea was as follows:
1. Resize the images to (224, 224)
2. Predict masks
3. Upsize each mask to it's original size


This idea was pretty okay to get a pipeline working, but the slices with non-equal heights and widths experienced a slight decrease in their score after upsizing, which in turn decreased the credibility of the validation scores during training.

Then I saw in the original [2.5D notebook](https://www.kaggle.com/code/awsaf49/uwmgi-2-5d-stride-2-data) the idea of pre-padding the slices so they are all the same dimension. I didn't know about it, and thought that it would produce the same results, but when I tested it, it made my validations scores more reliable, and it didn't mess up slices with different heights and widths.

To implemenet this idea, we need to provide a function which pads slices while loading them to a specific size, which should be either greater than all slices, or at-least the same the slices with the greatest size.

This function shall be inserted in `load_image` and `get_mask`

In [ ]:
def pad_img(img, up_size=None):
    if up_size is None:
        return img
    shape0 = np.array(img.shape[:2])
    resize = np.array(up_size)
    if np.any(shape0!=resize):
        diff = resize - shape0
        pad0 = diff[0]
        pad1 = diff[1]
        pady = [pad0//2, pad0//2 + pad0%2]
        padx = [pad1//2, pad1//2 + pad1%2]
        img = np.pad(img, [pady, padx])
        img = img.reshape((*resize))
    return img


def unpad_img(img, up_size, org_size):
    shape0 = np.array(org_size)
    resize = np.array(up_size)
    if np.any(shape0!=resize):
        diff = resize - shape0
        pad0 = diff[0]
        pad1 = diff[1]
        pady = [pad0//2, pad0//2 + pad0%2]
        padx = [pad1//2, pad1//2 + pad1%2]
        img = img[pady[0]:-pady[1], padx[0]:-padx[1], :]
        img = img.reshape((*shape0, 3))
    return img

    
def load_image(fname, up_size=None):
    img = np.array(Image.open(fname))
    img = np.interp(img, [np.min(img), np.max(img)], [0,255])
    return pad_img(img, up_size)

def get_25D_image(row, up_size=None):
    if up_size:
        imgs = np.zeros((*up_size, len(row['fnames'])))
    else:
        imgs = np.zeros((row['slice_h'], row['slice_w'], len(row['fnames'])))
        
    for i, fname in enumerate(row['fnames']):
        img = load_image(fname, up_size)
        imgs[..., i] += img
    return imgs.astype(np.uint8)
                   

def get_mask(row, up_size=None):
    if up_size:
        mask = np.zeros((*up_size, 3))
    else:
        mask = np.zeros((row['slice_h'], row['slice_w'], 3))
        
    if isinstance(row['large_bowel'], str):
        mask[..., 0] += pad_img(rle_decode(row['large_bowel'], shape=(row['slice_h'], row['slice_w']), color=255), up_size)
    if isinstance(row['small_bowel'], str):
        mask[..., 1] += pad_img(rle_decode(row['small_bowel'], shape=(row['slice_h'], row['slice_w']), color=255), up_size)
    if isinstance(row['stomach'], str):
        mask[..., 2] += pad_img(rle_decode(row['stomach'], shape=(row['slice_h'], row['slice_w']), color=255), up_size)
        
    return mask.astype(np.uint8)


#### Now what we only need to add the `up_size` argument with our preferred size, and let's see how that works out.

I'll use `partial` which sets an argument to be passed as default when the function is called.

In [ ]:
up_size = (320, 384)

tfms = [[partial(get_25D_image, up_size=up_size), PILImage.create],
        [partial(get_mask, up_size=up_size), PILMask.create]]
    
dsets = Datasets(train_df, tfms)

dls = dsets.dataloaders(bs=16, after_item=[ToTensor], after_batch=[IntToFloatTensor])

# Now let's see if our visualization works
dls.show_batch(nrows=4, ncols=4, max_n=16, figsize=(15, 15))

#### How about we use random resized crops (center crops for validations) with fastai augmentations?

In [ ]:
up_size = (320, 384)

tfms = [[partial(get_25D_image, up_size=up_size), PILImage.create],
        [partial(get_mask, up_size=up_size), PILMask.create]]

splits = RandomSplitter()(train_df)
dsets = Datasets(train_df, tfms, splits=splits)

dls = dsets.dataloaders(bs=16, after_item=[RandomResizedCrop(224), ToTensor], after_batch=[IntToFloatTensor, *aug_transforms(mult=0.8)])

# Now let's see if our visualization works
dls.show_batch(nrows=4, ncols=4, max_n=16, figsize=(15, 15))

In [ ]:
# Check validation set
dls.valid.show_batch(nrows=4, ncols=4, max_n=16, figsize=(15, 15))

<a id='data-six'></a>
## 2.5D Data with albumentations

The key to using Albumentations in fastai is to wrap the albumentation pipeline you want in a fastai `Transform`. I won't get into detail with this, but I'll provide a reciepe that you can customize however you like.

First let's make two functions, one that gets the training augmentations, and another that get the validation augmentation.

In [ ]:
import albumentations as A

def get_train_aug(img_size, crop=0.9, p=0.4):
    crop_size = round(img_size[0]*crop)
    return A.Compose([
            A.RandomCrop(height=crop_size, width=crop_size, always_apply=True),
            A.HorizontalFlip(p=p),
            A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
            ], p=p),
            A.CoarseDropout(
                max_holes=8, min_holes=8,
                max_height=crop_size//10, max_width=crop_size//10,
                min_height=4, min_width=4, mask_fill_value=0, p=0.2*p),
            A.ShiftScaleRotate(
                shift_limit=0.0625, scale_limit=0.2, rotate_limit=25,
                interpolation=cv2.INTER_AREA, p=p),
            A.HorizontalFlip(p=0.5*p),
            A.OneOf([
                A.MotionBlur(p=0.2*p),
                A.MedianBlur(blur_limit=3, p=0.1*p),
                A.Blur(blur_limit=3, p=0.1*p),
            ], p=0.2*p),
            A.GaussNoise(var_limit=0.001, p=0.2*p),
            A.OneOf([
                A.OpticalDistortion(p=0.3*p),
                A.GridDistortion(p=0.1*p),
                A.PiecewiseAffine(p=0.3*p),
            ], p=0.2*p),
            A.OneOf([
                A.Sharpen(p=0.2*p),
                A.Emboss(p=0.2*p),
                A.RandomBrightnessContrast(p=0.2*p),
            ]),
        ])


def get_test_aug(img_size, crop=0.9):
    crop_size = round(crop*img_size[0])
    return  A.Compose([
        A.CenterCrop(height=crop_size, width=crop_size),
    ])

Next we'll make `Transform` object that wraps our augmentation and makes it usable within the fastai low-level api pipeline.

In [ ]:
import cv2

class AlbumentationsTransform(ItemTransform, RandTransform):
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx): self.idx = split_idx
    
    def encodes(self, x):
        if len(x) > 1:
            img, mask = x
            if self.idx == 0:
                aug = self.train_aug(image=np.array(img), mask=np.array(mask))    
            else:
                aug = self.valid_aug(image=np.array(img), mask=np.array(mask))
            return PILImage.create(aug["image"]), PILMask.create(aug["mask"])
        else:
            img = x[0]
            aug = self.valid_aug(image=np.array(img))
            return PILImage.create(aug["image"])

And now let's test it.

In [ ]:
# Let's take a sample of the training dataframe since we aren't going to use it right now 
# in anything more than demonstration

dev_df = train_df.sample(frac=0.2)


up_size = (320, 384)

tfms = [[partial(get_25D_image, up_size=up_size), PILImage.create],
        [partial(get_mask, up_size=up_size), PILMask.create]]

splits = RandomSplitter()(dev_df)
dsets = Datasets(train_df, tfms, splits=splits)

albu_aug = AlbumentationsTransform(get_train_aug(up_size), get_test_aug(up_size))

dls = dsets.dataloaders(bs=16, after_item=[albu_aug, ToTensor], after_batch=[IntToFloatTensor(div_mask=255), 
                                                                             Normalize.from_stats(*imagenet_stats)])

# Now let's see if our visualization works
dls.show_batch(nrows=4, ncols=4, max_n=16, figsize=(15, 15))

<a id='section-two'></a>
# Training

<a id='train-one'></a>
## Metrics and Loss Functions

To save ourselves some hassle, I'll here the metrics and loss functions.

### Metrics

In [ ]:
from scipy.spatial.distance import directed_hausdorff


def dice_coeff_adj(inp, targ):
    inp = np.where(sigmoid(inp).cpu().detach().numpy() > 0.5, 1, 0)
    targ = targ.cpu().detach().numpy()
    eps = 1e-5
    dice_scores = []
    for i in range(targ.shape[0]):
        dice_i = []
        for j in range(targ.shape[1]):
            if inp[i, j].sum() == targ[i, j].sum() == 0:
                continue
            I = (targ[i, j] * inp[i, j]).sum()
            U =  targ[i, j].sum() + inp[i, j].sum()
            dice_i.append((2.*I)/(U+eps))
        if dice_i:
            dice_scores.append(np.mean(dice_i))
    
    if dice_scores:
        return np.mean(dice_scores)
    else:
        return 0
    
    
def hd_dist_per_slice(inp, targ, seed):    
    inp = np.argwhere(inp) / np.array(inp.shape)
    targ = np.argwhere(targ) / np.array(targ.shape)
    haussdorf_dist = 1 - directed_hausdorff(inp, targ, seed)[0]
    return haussdorf_dist if haussdorf_dist > 0 else 0

def hd_dist_adj(inp, targ, seed=42):
    inp = np.where(sigmoid(inp).cpu().detach().numpy() > 0.5, 1, 0)
    targ = targ.cpu().detach().numpy()
    hd_scores = []
    for i in range(targ.shape[0]):
        hd_i = []
        for j in range(targ.shape[1]):
            if inp[i, j].sum() == targ[i, j].sum() == 0:
                continue
            hd_i.append(hd_dist_per_slice(inp[i, j], targ[i, j], seed))
        if hd_i:
            hd_scores.append(np.mean(hd_i))
    if hd_scores:
        return np.mean(hd_scores)
    else:
        return 0

def custom_metric_adj(inp, targ, seed=42):
    hd_score_per_batch = hd_dist_adj(inp, targ, seed)
    dice_score_per_batch = dice_coeff_adj(inp, targ)
    
    return 0.4*dice_score_per_batch + 0.6*hd_score_per_batch

### Loss Functions

In [ ]:
# Loss functions
class DiceBCEModule(Module):
    def __init__(self, eps:float=1e-5, from_logits=True):
        store_attr()
        
    def forward(self, inp:Tensor, targ:Tensor) -> Tensor:
        inp = inp.view(-1)
        targ = targ.view(-1)
        
        if self.from_logits: 
            bce_loss = nn.BCEWithLogitsLoss()(inp, targ)
            inp = torch.sigmoid(inp)
            
            
        intersection = (inp * targ).sum()                            
        dice = (2.*intersection + self.eps)/(inp.sum() + targ.sum() + self.eps)  
        
        return 0.5*(1 - dice) + 0.5*bce_loss


class DiceBCELoss(BaseLoss):
    def __init__(self, *args, eps:float=1e-5, from_logits=True, thresh=0.5, **kwargs):
        super().__init__(DiceBCEModule, *args, eps=eps, from_logits=from_logits, flatten=False, is_2d=True, floatify=True, **kwargs)
        self.thresh = thresh
    
    def decodes(self, x:Tensor) -> Tensor:
        "Converts model output to target format"
        return (x>self.thresh).long()

    def activation(self, x:Tensor) -> Tensor:
        "`nn.BCEWithLogitsLoss`'s fused activation function applied to model output"
        return torch.sigmoid(x)

# Source: https://www.kaggle.com/code/thedrcat/focal-multilabel-loss-in-pytorch-explained/notebook
def focal_binary_cross_entropy(logits, targets, gamma=2, n=3):
    p = torch.sigmoid(logits)
    p = torch.where(targets >= 0.5, p, 1-p)
    logp = - torch.log(torch.clamp(p, 1e-4, 1-1e-4))
    loss = logp*((1-p)**gamma)
    loss = n*loss.mean()
    return loss

class DiceFocalModule(Module):
    def __init__(self, eps:float=1e-5, from_logits=True, ws=[0.5, 0.5], gamma=2, n=3):
        store_attr()
        
    def forward(self, inp:Tensor, targ:Tensor) -> Tensor:
        inp = inp.view(-1)
        targ = targ.view(-1)
        
        if self.from_logits: 
            focal_loss = focal_binary_cross_entropy(inp, targ, self.gamma, self.n)
            inp = torch.sigmoid(inp)
            
            
        intersection = (inp * targ).sum()                            
        dice = (2.*intersection + self.eps)/(inp.sum() + targ.sum() + self.eps)  
        
        return self.ws[0]*(1 - dice) + self.ws[1]*focal_loss
    
class DiceFocalLoss(BaseLoss):
    def __init__(self, *args, eps:float=1e-5, from_logits=True, ws=[0.5, 0.5], gamma=2, n=3, thresh=0.5, **kwargs):
        super().__init__(DiceFocalModule, *args, eps=eps, from_logits=from_logits, ws=ws, gamma=gamma, n=n, flatten=False, is_2d=True, floatify=True, **kwargs)
        self.thresh = thresh
    
    def decodes(self, x:Tensor) -> Tensor:
        "Converts model output to target format"
        return (x>self.thresh).long()

    def activation(self, x:Tensor) -> Tensor:
        "`nn.BCEWithLogitsLoss`'s fused activation function applied to model output"
        return torch.sigmoid(x)

class FocalTverskyLossModule(Module):
    def __init__(self, eps:float=1e-5, from_logits=True, alpha=0.3, beta=0.7, gamma=3/4):
        store_attr()
        
    def forward(self, inp:Tensor, targ:Tensor) -> Tensor:
        inp = inp.view(-1)
        targ = targ.view(-1)
        
        if self.from_logits: 
            inp = torch.sigmoid(inp)
            
        inp_0, inp_1 = inp, 1 - inp
        targ_0, targ_1 = targ, 1 - targ
            
        num = (inp_0 * targ_0).sum() 
        denom = num + (self.alpha * (inp_0 * targ_1).sum()) + (self.beta * (inp_1 * targ_0).sum()) + self.eps
        loss = 1 - (num / denom)
        return loss**self.gamma 
    
class FocalTverskyLoss(BaseLoss):
    def __init__(self, *args, eps:float=1e-5, from_logits=True, alpha=0.3, beta=0.7, gamma=3/4, thresh=0.5, **kwargs):
        super().__init__(FocalTverskyLossModule, *args, eps=eps, from_logits=from_logits, alpha=alpha, beta=beta, gamma=gamma, flatten=False, is_2d=True, floatify=True, **kwargs)
        self.thresh = thresh
    
    def decodes(self, x:Tensor) -> Tensor:
        "Converts model output to target format"
        return (x>self.thresh).long()

    def activation(self, x:Tensor) -> Tensor:
        "`nn.BCEWithLogitsLoss`'s fused activation function applied to model output"
        return torch.sigmoid(x)

def focal_binary_cross_entropy(logits, targets, gamma=2, n=3):
    p = torch.sigmoid(logits)
    p = torch.where(targets >= 0.5, p, 1-p)
    logp = - torch.log(torch.clamp(p, 1e-4, 1-1e-4))
    loss = logp*((1-p)**gamma)
    loss = n*loss.mean()
    return loss

class ComboModule(Module):
    def __init__(self, eps:float=1e-5, from_logits=True, ws=[2, 3, 1], gamma=2, n=3):
        store_attr()
        
    def forward(self, inp:Tensor, targ:Tensor) -> Tensor:
        inp = inp.view(-1)
        targ = targ.view(-1)
        
        if self.from_logits: 
            focal_loss = focal_binary_cross_entropy(inp, targ, self.gamma, self.n)
            bce_loss = nn.BCEWithLogitsLoss()(inp, targ)
            inp = torch.sigmoid(inp)
                
        intersection = (inp * targ).sum()                            
        dice = (2.*intersection + self.eps)/(inp.sum() + targ.sum() + self.eps)  
        
        return self.ws[0]*(1 - dice) + self.ws[1]*focal_loss + self.ws[2]*bce_loss
    
class ComboLoss(BaseLoss):
    def __init__(self, *args, eps:float=1e-5, from_logits=True, ws=[2, 3, 1], gamma=2, n=3, thresh=0.5, **kwargs):
        super().__init__(ComboModule, *args, eps=eps, from_logits=from_logits, ws=ws, gamma=gamma, n=n, flatten=False, is_2d=True, floatify=True, **kwargs)
        self.thresh = thresh
    
    def decodes(self, x:Tensor) -> Tensor:
        "Converts model output to target format"
        return (x>self.thresh).long()

    def activation(self, x:Tensor) -> Tensor:
        "`nn.BCEWithLogitsLoss`'s fused activation function applied to model output"
        return torch.sigmoid(x)

<a id='train-two'></a>
## Using SMP models with Fastai

Now that we have a decent data preparation pipeline, let's make use of smp's amazing library of models to build us a training pipeline.

First let's make a function that returns a Unet with our preferred encoder.

In [ ]:
!pip install segmentation_models_pytorch

In [ ]:
import segmentation_models_pytorch as smp

def build_model(encoder_name, in_c=3, classes=3, weights="imagenet"):
    model = smp.Unet(
        encoder_name=encoder_name,      
        encoder_weights=weights,     
        in_channels=in_c,                
        classes=classes,        
        activation=None
    )
    return model

Now I'll provide a function that splits any of model parameters from smp into encoder and decoder, so that we can freeze and unfreeze encoder layers.

In [ ]:
def smp_splitter(model):
    model_layers = list(model.children())
    encoder_params = params(model_layers[0])
    decoder_params = params(model_layers[1]) + params(model_layers[2])
    return L(encoder_params, decoder_params)

And now let's use the latest dataloader to create a training pipeline using `Learner`

In [ ]:
model = build_model('efficientnet-b0')
metrics = [dice_coeff_adj, hd_dist_adj, custom_metric_adj]
loss_func = ComboLoss()
splitter = smp_splitter

In [ ]:
learn = Learner(dls, model, metrics=metrics, loss_func=loss_func, splitter=splitter).to_fp16()
learn.freeze()

#### Let's fine tune this model for 2 epochs, one frozen and another unfrozen. This isn't suppossed to produce world class results, but it's just a demonstration.

In [ ]:
learn.fine_tune(1, 1e-2)

In [ ]:
learn.export('test_model.pkl')

You can customize this basic recipe to your liking including:
1. Image size and augmentations
3. Loss function
4. Model architecture and encoder
5. Training scheme

The possibilities are endless. 

<a id="train-three"></a>
## Using Fastai's DynamicUnet with timm encoder?

In theory, you can pass any encoder to `DyanmicUnet` and it should work, but after some trials, I found out that unfortunatley, it doesn't work properly, so I made some adjustments to the original code to make it work.

What's cool about fastai's `DynamicUnet` is that it has some configurations that aren't available in other UNet implementations. For example:
1. You can add self attentions to the Unet Decoder blocks
2. You can customize the activations function in the Unet Decoder blocks
3. Unet Decoder blocks use PixelShuffle upsampling instead of regular upsampling

In [ ]:
import timm

def timm_model_sizes(encoder, img_size):
    sizes = []
    for layer in encoder.feature_info:
        sizes.append(torch.Size([1, layer['num_chs'], img_size[0]//layer['reduction'], img_size[1]//layer['reduction']]))
    return sizes


def get_timm_output_layers(encoder):
    outputs = []
    for layer in encoder.feature_info:
        # Converts 'blocks.0.0' to ['blocks', '0', '0']
        attrs = layer['module'].split('.')
        output_layer = getattr(encoder, attrs[0])[int(attrs[1])][int(attrs[2])]
        outputs.append(output_layer)
    return outputs


class DynamicTimmUnet(SequentialEx):
    "Create a U-Net from a given architecture in timm."
    def __init__(self, encoder, n_out, img_size, blur=False, blur_final=True, self_attention=False,
                 y_range=None, last_cross=True, bottle=False, act_cls=defaults.activation,
                 init=nn.init.kaiming_normal_, norm_type=None, **kwargs):
        imsize = img_size
        sizes = timm_model_sizes(encoder, img_size)
        sz_chg_idxs = list(reversed(range(len(sizes))))
        outputs = list(reversed(get_timm_output_layers(encoder)))
        self.sfs = hook_outputs(outputs, detach=False)
        
        # cut encoder
        encoder = nn.Sequential(*list(encoder.children()))[:-5]
        
        x = dummy_eval(encoder, imsize).detach()

        ni = sizes[-1][1]
        middle_conv = nn.Sequential(ConvLayer(ni, ni*2, act_cls=act_cls, norm_type=norm_type, **kwargs),
                                    ConvLayer(ni*2, ni, act_cls=act_cls, norm_type=norm_type, **kwargs)).eval()
        x = middle_conv(x)
        layers = [encoder, BatchNorm(ni), nn.ReLU(), middle_conv]

        for i,idx in enumerate(sz_chg_idxs):
            not_final = i!=len(sz_chg_idxs)-1
            up_in_c, x_in_c = int(x.shape[1]), int(sizes[idx][1])
            do_blur = blur and (not_final or blur_final)
            sa = self_attention and (i==len(sz_chg_idxs)-3)
            unet_block = UnetBlock(up_in_c, x_in_c, self.sfs[i], final_div=not_final, blur=do_blur, self_attention=sa,
                                   act_cls=act_cls, init=init, norm_type=norm_type, **kwargs).eval()
            layers.append(unet_block)
            x = unet_block(x)

        ni = x.shape[1]
        if imsize != sizes[0][-2:]: layers.append(PixelShuffle_ICNR(ni, act_cls=act_cls, norm_type=norm_type))
        layers.append(ResizeToOrig())
        if last_cross:
            layers.append(MergeLayer(dense=True))
            ni += in_channels(encoder)
            layers.append(ResBlock(1, ni, ni//2 if bottle else ni, act_cls=act_cls, norm_type=norm_type, **kwargs))
        layers += [ConvLayer(ni, n_out, ks=1, act_cls=None, norm_type=norm_type, **kwargs)]
        apply_init(nn.Sequential(layers[3], layers[-2]), init)
        #apply_init(nn.Sequential(layers[2]), init)
        if y_range is not None: layers.append(SigmoidRange(*y_range))
        layers.append(ToTensorBase())
        super().__init__(*layers)

    def __del__(self):
        if hasattr(self, "sfs"): self.sfs.remove()
            
            
def dynamic_unet_splitter(model):
    return L(model[0], model[1:]).map(params)


#### Now let's do this.

In [ ]:
dls = dsets.dataloaders(bs=16, after_item=[albu_aug, ToTensor],
                        after_batch=[IntToFloatTensor(div_mask=255), Normalize.from_stats(*imagenet_stats)])

The training scheme I'll use right now is inspired by this [lesson](https://walkwithfastai.com/Segmentation) for walk with fastai

In [ ]:
img_size = [round(0.9*320) for _ in range(2)]

encoder = timm.create_model('efficientnet_b0', pretrained=True)

# Let's use self attentions and Mish activation function 
model = DynamicTimmUnet(encoder, 3, img_size, self_attention=True, act_cls=Mish)

# We'll also use ranger optimizer with is RAdam with Lookahead
learn = Learner(dls, model, metrics=metrics, loss_func=loss_func, splitter=dynamic_unet_splitter, opt_func=ranger).to_fp16()
learn.freeze()

In [ ]:
learn.lr_find()

In [ ]:
# Let's also use flat cosine annealing lr shceduler
lr = 1e-3
learn.fit_flat_cos(1, lr)

Let's unfreeze the encoder layers and train with discriminative learning rates.

In [ ]:
learn.unfreeze()
learn.fit_flat_cos(2, slice(lr/400, lr/4))

<a id='infer'></a>
# Inference

### Okay let's say we have trained a model and we want to submit, what should we do?
1. We should make a test dataframe that we can use similar to the train datafrrame
1. We should predict using the full size of padded image or we can crop them if we want 
2. Then we should unpad the predicted masks to their specific sizes
3. And we should convert the masks to rle format 
4. Finally we make our submission file


*Note that in this notebook, the internet is enabled, while in real inference notebook the internet should be disabled, which could be a problem while importing `smp` and `timm`, but there is a work around where you can load these packages as datasets, then install them directly without internet.*

### Make a test dataframe that we can use similar to the train datafrrame

In [ ]:
def create_df(df, fnames):
    df = df.copy()
    df = df.pivot(index='id', columns='class', values='segmentation').reset_index()
    
    df['partial_fname'] = df.id
    fname_df = pd.DataFrame({'partial_fname': [f'{fname.parts[-3]}_slice_{fname.parts[-1][6:10]}' for fname in fnames],
                             'fname': fnames})

    df = df.merge(fname_df, on='partial_fname').drop('partial_fname', axis=1)

    df['case_id'] = df.id.apply(lambda x: x.split('_')[0])
    df['day_num'] = df.id.apply(lambda x: x.split('_')[1])

    df['slice_w'] = df["fname"].apply(lambda x: int(str(x)[:-4].rsplit("_",4)[1]))
    df['slice_h'] = df["fname"].apply(lambda x: int(str(x)[:-4].rsplit("_",4)[2]))
    
    channels = 3
    stride = 2
    for j, i in enumerate(range(-1*(channels-channels//2-1), channels//2+1)):
        method = 'ffill'
        if i <= 0: method = 'bfill'
        df[f'fname_{j:02}'] = df.groupby(['case_id', 'day_num'])['fname'].shift(stride*-i).fillna(method=method)

    df['fnames'] = df[[f'fname_{j:02d}' for j in range(channels)]].values.tolist()
    
    return df

In [ ]:
data_path = '../input/uw-madison-gi-tract-image-segmentation/'

train_path = Path(data_path+'train')
test_path = Path(data_path+'test')

train_fnames = get_image_files(train_path)
test_fnames = get_image_files(test_path)

sample_submission = pd.read_csv(data_path+'sample_submission.csv')

if sample_submission.shape[0] > 0: 
    test = sample_submission.copy()
else:
    test_fnames = train_fnames
    test_path = train_path
    test = train.copy()
    test = test.sample(frac=1.0, random_state=42)

test_df = create_df(test, test_fnames)

### Prediction and conversion to rle masks

### Load learner

In [ ]:
learn = load_learner('test_model.pkl')

### Create test dataloader

In [ ]:
# I'll sample the test set here to make a quick demonstartion
test_df = test_df.sample(frac=0.1)

In [ ]:
bs = learn.dls.bs
test_dl = learn.dls.test_dl(test_df, bs=bs, shuffle=False).to('cuda')

### Predict and convert to rle masks

In [ ]:
def mask2rle(mask):
    """
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    """
    mask = np.array(mask)
    pixels = mask.flatten()
    pad = np.array([0])
    pixels = np.concatenate([pad, pixels, pad])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]

    return " ".join(str(x) for x in runs)

def resize_img_to_org_size(img, org_size):
    shape0 = np.array(img.shape[:2])
    diff = org_size - shape0
    if np.any(diff < 0):
        img = pad_img_nc(img, (320, 384))
        resized = unpad_img_nc(img, org_size)
    else:
        resized = pad_img_nc(img, org_size)
    return resized

def get_rle_masks(preds, df):
    rle_masks = []
    for pred, width, height in zip(preds, df['slice_w'], df['slice_h']):
        upsized_mask = resize_img_to_org_size(pred, (height, width))
        for i in range(3):
            rle_mask = mask2rle(upsized_mask[:, :, i])
            rle_masks.append(rle_mask)
    return rle_masks

def unpad_img_nc(img, org_size):
    shape0 = np.array(org_size)
    resize = np.array(img.shape[:2])
    if np.any(shape0!=resize):
        diff = resize - shape0
        pad0 = diff[0]
        pad1 = diff[1]
        pady = [pad0//2, pad0//2 + pad0%2]
        padx = [pad1//2, pad1//2 + pad1%2]
        
        if pady[0] != 0:
            img = img[pady[0]:-pady[1], :, :]
            
        if padx[0] != 0:
            img = img[:, padx[0]:-padx[1], :]
            
        img = img.reshape((*shape0, img.shape[-1]))
    return img

def pad_img_nc(img, up_size=None):
    if up_size is None:
        return img
    shape0 = np.array(img.shape[:2])
    resize = np.array(up_size)
    if np.any(shape0!=resize):
        diff = resize - shape0
        pad0 = diff[0]
        pad1 = diff[1]
        pady = [pad0//2, pad0//2 + pad0%2]
        padx = [pad1//2, pad1//2 + pad1%2]
        padz = [0, 0]
        img = np.pad(img, [pady, padx, padz])
        img = img.reshape((*resize, img.shape[-1]))
    return img

In [ ]:
from tqdm import tqdm
import gc

learn.model = learn.model.cuda()
learn.model.eval()
masks = []

with torch.no_grad():
    for i, b in enumerate(tqdm(test_dl)):
        b.to('cuda')
        b_preds = (sigmoid(learn.model(b)) > 0.5).permute(0, 2, 3, 1).cpu().detach().numpy().astype(np.uint8)

        masks.extend(get_rle_masks(b_preds, test_df.iloc[i*bs:i*bs+bs]))

        # test_preds[i*bs:i*bs+bs] = b_preds
        del b_preds
        torch.cuda.empty_cache()
        gc.collect()

### Make submission file

In [ ]:
def get_case_id(fname):
    return fname.parts[5] + '_' + fname.parts[7][:10]

In [ ]:
from itertools import chain

submission = pd.DataFrame({
        'id': chain.from_iterable([[get_case_id(fname)]*3 for fname in test_df['fname']]),
        'class': chain.from_iterable([['large_bowel', 'small_bowel', 'stomach'] for _ in test_df['fname']]),
        'predicted': masks,
    })
    

# Merge with sample submission to preserve order to slices during scoring and avoid 0 scores
if sample_submission.shape[0] > 0:
    del sample_submission['segmentation']
    submission = sample_submission.merge(submission, on=['id', 'class'])

submission.to_csv('submission.csv', index=False)

#### And that's how you do inference.

#### I highly recommend that you fork this notebook and start tinkering around with the code provided, as you'll understand more about than I could ever try to tell you in words. Morever take a deep dive into [fastai documentation](docs.fast.ai) tutorials section.

#### Also if you are unware with fastai, I recommend that you check out there course, and their book which in my opinion is the best way around to get into deep learning due to it's application driven approach.

#### I'm not a great writer and I don't go into much detail, but if anyone wanted to clarify on any piece of code I'd be happy to walk them through how it works.

#### Next up
   - Using more than 3 channels in 2.5D Data

#### This is a work in progress. Don't forget to upvote if you liked this notebook so far.